In [11]:
import torchvision
import numpy as np
import matplotlib
import wandb
from src import preprocess, activation_functions, initializers, layer, loss_functions, optimizers, FFNN, training, wandblogger, dropout, batchnorm, sweeping
import nbformat


In [12]:
train_dataset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=False)
test_dataset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=False)

In [13]:
validation_percent = 0.2
one_hot_encoder = preprocess.OneHotEncoder()


train_size = int((1 - validation_percent) * len(train_dataset))
validation_size = len(train_dataset) - train_size

x_train = train_dataset.data[:train_size].reshape(train_size, -1)
targets_train = np.array(train_dataset.targets[:train_size])
y_train = one_hot_encoder.transform(targets_train)

x_valid = train_dataset.data[train_size:].reshape(validation_size, -1)
targets_valid = np.array(train_dataset.targets[train_size:])
y_valid = one_hot_encoder.transform(targets_valid)

x_test = test_dataset.data.reshape(len(test_dataset), -1)
targets_test = np.array(test_dataset.targets)
y_test = one_hot_encoder.transform(targets_test)

print(type(x_train), type(y_train))
print(x_train.shape, y_train.shape)

<class 'torch.Tensor'> <class 'numpy.ndarray'>
torch.Size([48000, 784]) (48000, 10)


C:\Users\Ludvig\AppData\Local\Temp\ipykernel_43264\3457253408.py:9: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_train = np.array(train_dataset.targets[:train_size])
C:\Users\Ludvig\AppData\Local\Temp\ipykernel_43264\3457253408.py:13: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  targets_valid = np.array(train_dataset.targets[train_size:])
C:\Users\Ludvig\AppData\Local\Temp\ipykernel_43264\3457253408.py:17: DeprecationWarning: __array__ implementation doesn't accep

In [14]:
#find min and max value of y_train
print("y_train min:", np.min(y_train), "y_train max:", np.max(y_train))

y_train min: 0.0 y_train max: 1.0


In [15]:
project = "Fashion-MNIST"
entity = "DTU-Deep-Learning-Project"

In [16]:
sweep_config = {
    "method": "random",
    "metric": {"name": "val_loss", "goal": "minimize"},
    "parameters": {
        "optimizer": {"values": ["sgd"]},
        "learning_rate": {"min": 0.0005, "max": 0.1},
        "batch_size": {"values": [32, 64, 128]},
        "epochs": {"value": 20},
        "normalize_data": {"values": [True, False]},
        # 🔥 The important part:
        # Hidden layer configurations can vary
        "layer_sizes": {
            "values": [
                [784, 512, 256],
                [784, 512, 512, 256, 128]
            ]
        },
        
        "use_batchnorm": {"values": [True, False]},
        "dropout_rate": {"values": [0.2, 0.35, 0.5]},
    },
}

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity=entity,
    project=project)

wandb.agent(sweep_id, function=lambda: sweeping.train_sweep(entity, project, config=sweep_config, x_train=x_train, y_train=y_train, x_valid=x_valid, y_valid=y_valid), count=10)

Create sweep with ID: 6zg354uy
Sweep URL: https://wandb.ai/DTU-Deep-Learning-Project/Fashion-MNIST/sweeps/6zg354uy


wandb: Agent Starting Run: d3gecr8z with config:
wandb: 	batch_size: 64
wandb: 	dropout_rate: 0.35
wandb: 	epochs: 20
wandb: 	layer_sizes: [784, 512, 256]
wandb: 	learning_rate: 0.06398985515514814
wandb: 	normalize_data: True
wandb: 	optimizer: sgd
wandb: 	use_batchnorm: False
